<a href="https://colab.research.google.com/github/FischerMoseley/ball_boi/blob/main/BallBotInfo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

# Install drake (and underactuated).
if 'google.colab' in sys.modules and importlib.util.find_spec('underactuated') is None:
    urlretrieve(f"http://underactuated.csail.mit.edu/scripts/setup/setup_underactuated_colab.py",
                "setup_underactuated_colab.py")
    from setup_underactuated_colab import setup_underactuated
    setup_underactuated(underactuated_sha='3e23b46200d113f9432c17b17751e16e7b42c995', drake_version='0.27.0', drake_build='release')

# Setup matplotlib.
from IPython import get_ipython
if get_ipython() is not None: get_ipython().run_line_magic("matplotlib", "inline")

# python libraries
import numpy as np
from IPython.display import HTML, display
from matplotlib import pyplot
import pydrake.all
from pydrake.all import (
    DiagramBuilder, LinearQuadraticRegulator, Saturation, SceneGraph, Simulator, MakeVectorVariable, Variable,
    WrapToSystem, AddMultibodyPlantSceneGraph, Parser, MathematicalProgram,MultibodyPlant, PlanarSceneGraphVisualizer, MultibodyForces_,
    Expression,DecomposeLumpedParameters, VectorSystem, LogOutput,DiscreteAlgebraicRiccatiEquation,  TemplateSystem, LeafSystem_, BasicVector_, 
    DiagramBuilder, Simulator_, LeafSystem, BasicVector, Integrator
)

from underactuated.jupyter import AdvanceToAndVisualize, SetupMatplotlibBackend, running_as_notebook
import numpy as np
from ipywidgets import FloatSlider, ToggleButton
from IPython.display import display, SVG
import pydot

# underactuated imports
from underactuated import FindResource, ManipulatorDynamics

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import scipy.io

So we are going to see if we can generate the graphs we need to,

In [ ]:

def cost_breakdown(q, u, goal, T, display=False, yscale="linear"):
  total_distance_from_target = 0
  total_actuation = 0
  goal_x = goal[0]

  distance_from_target = np.array([(q[t][0]**2 + goal_x**2)**0.5 for t in range(len(q))])
  actuation = np.array([u[t][0] for t in range(len(q))])  # assuming u_x = u_theta since that's what Jorge said

  distance_squared = distance_from_target**2
  actuation_squared = actuation**2
  cost = distance_squared + actuation_squared
  total_cost = np.sum(cost)

  if display:
    cost_sum = np.zeros(cost.shape)
    total = 0
    for i in range(cost.shape[0]):
      total += cost[i]
      cost_sum[i] = total

    time_steps = len(q)
    increment = T/time_steps
    times = [t*increment for t in range (len(q))]
    plt.plot(times, distance_from_target, label="Distance From Target")
    plt.plot(times, actuation, label="Actuation")
    plt.plot(times, cost, label="Cost")
    plt.plot(times, cost_sum, label="Total Cost")
    plt.yscale(yscale)
    plt.legend()
    plt.show()

  return distance_from_target, actuation, cost, total_cost

def settling_time(q, u, goal, T, epsilon=1e-3):
  """ Return None if never settled """
  settling_time = None
  goal_x = goal[0]

  for t in range(q.shape[0] - 1, -1, -1):
    cur_x = q[t][0]
    if abs(cur_x - goal_x) > epsilon:  # meaning, they're unequal
      settling_time = t + 1
      break

  if abs(q[-1][0] - goal_x) > epsilon:
    return None

  time_steps = len(q)
  increment = T/time_steps
  settling_time *= increment
  return settling_time

def overshoot(q, u, goal, T, epsilon=1e-3, display=False):
  """ Return None if never reached goal """
  settling_time = None
  goal_x = goal[0]
  start_x = q[0][0]
  timestep_reached_goal = None

  if start_x < goal_x:
    for t in range(len(q)):
      cur_x = q[t][0]
      if cur_x + epsilon >= goal_x:
        timestep_reached_goal = t  # overestimate if not exactly equal
        break
  else:
    for t in range(len(q)):
      cur_x = q[t][0]
      if cur_x - epsilon <= goal_x:
        timestep_reached_goal = t  # overestimate if not exactly equal
        break

  total_overshoot = 0
  for t in range(timestep_reached_goal, len(q)):
    cur_x = q[t][0]
    total_overshoot += abs(goal_x - cur_x)

  if display:
    # here, I'll plot the distance from goal as a function of time
    xs = np.array([q[t][0] for t in range(len(q))])
    distances = xs - goal_x
    time_steps = len(q)
    increment = T/time_steps
    times = [t*increment for t in range(len(q))]
    plt.plot(times, distances)
    plt.title("Distance from Goal")
    plt.xlabel("Time (s)")
    plt.ylabel("Distance")
    plt.show()

  return total_overshoot/len(q)  # can def change this

Now we'll define the 3 different Urdifs we'll use

In [ ]:
ballbot_standard = """
<?xml version="1.0"?>

<robot xmlns="http://drake.mit.edu"
 xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
 name="BallBot">

  <link name="ground">
    <visual>
      <origin xyz="0 0 -5" rpy="0 0 0" />
      <geometry>
        <box size="1000 1000 10" />
      </geometry>
      <material>
        <color rgba="0.93 .74 .4 1" />
      </material>
    </visual>
  </link>

  <joint name="ground_weld" type="fixed">
    <parent link="world" />
    <child link="ground" />
  </joint>

  <link name="ball">
    <inertial>
      <origin xyz="0 0 0" rpy="0 0 0" />
      <mass value="5" />
      <inertia ixx=".2" ixy="0" ixz="0" iyy="0.2" iyz="0" izz="0.2" />
    </inertial>
    <visual>
      <origin xyz="0 0 0" rpy="0 0 0" />
      <geometry>
        <sphere radius=".1" />
      </geometry>
      <material>
        <color rgba="0.25 0.52 0.96 1" />
      </material>
    </visual>
  </link>

  <link name="bot">
    <inertial>
      <origin xyz="0 0 .05" rpy="0 0 0" />
      <mass value="4" />
      <inertia ixx="0.10" ixy="0" ixz="0" iyy="0.18" iyz="0" izz="0.1" />
    </inertial>
    <visual>
      <origin xyz="0 0 .05" rpy="0 0 0" />
      <geometry>
         <cylinder length=".1" radius=".12" />
      </geometry>
      <material>
        <color rgba=".61 .63 .67 1" />
      </material>
    </visual>
  </link>
  
  <joint name="x" type="prismatic">
    <parent link="world" />
    <child link="ball" />
    <origin xyz="0 0 .1" />
    <axis xyz="1 0 0" />
    <dynamics damping="0.1" />
  </joint>
   

  <joint name="theta1" type="continuous">
    <parent link="ball" />
    <child link="bot" />
    <origin xyz="0 0 0" />
    <axis xyz="0 1 0" />
    <dynamics damping="0.1" />
  </joint>


  <transmission type="SimpleTransmission" name="ball_torque">
    <actuator name="torque" />
    <joint name="theta1" />
    <mechanicalReduction>1</mechanicalReduction>
  </transmission>

  <transmission type="SimpleTransmission" name="ball_force">
    <actuator name="force" />
    <joint name="x" />
    <mechanicalReduction>.1</mechanicalReduction>
  </transmission>
</robot>
"""
#here the ball is 20% heavier and has 20% more moment of inertia
ballbot_heavyball = """
<?xml version="1.0"?>

<robot xmlns="http://drake.mit.edu"
 xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
 name="BallBot">

  <link name="ground">
    <visual>
      <origin xyz="0 0 -5" rpy="0 0 0" />
      <geometry>
        <box size="1000 1000 10" />
      </geometry>
      <material>
        <color rgba="0.93 .74 .4 1" />
      </material>
    </visual>
  </link>

  <joint name="ground_weld" type="fixed">
    <parent link="world" />
    <child link="ground" />
  </joint>

  <link name="ball">
    <inertial>
      <origin xyz="0 0 0" rpy="0 0 0" />
      <mass value="6" />
      <inertia ixx=".24" ixy="0" ixz="0" iyy="0.24" iyz="0" izz="0.24" />
    </inertial>
    <visual>
      <origin xyz="0 0 0" rpy="0 0 0" />
      <geometry>
        <sphere radius=".1" />
      </geometry>
      <material>
        <color rgba="0.25 0.52 0.96 1" />
      </material>
    </visual>
  </link>

  <link name="bot">
    <inertial>
      <origin xyz="0 0 .05" rpy="0 0 0" />
      <mass value="4" />
      <inertia ixx="0.10" ixy="0" ixz="0" iyy="0.18" iyz="0" izz="0.1" />
    </inertial>
    <visual>
      <origin xyz="0 0 .05" rpy="0 0 0" />
      <geometry>
         <cylinder length=".1" radius=".12" />
      </geometry>
      <material>
        <color rgba=".61 .63 .67 1" />
      </material>
    </visual>
  </link>
  
  <joint name="x" type="prismatic">
    <parent link="world" />
    <child link="ball" />
    <origin xyz="0 0 .1" />
    <axis xyz="1 0 0" />
    <dynamics damping="0.1" />
  </joint>
   

  <joint name="theta1" type="continuous">
    <parent link="ball" />
    <child link="bot" />
    <origin xyz="0 0 0" />
    <axis xyz="0 1 0" />
    <dynamics damping="0.1" />
  </joint>


  <transmission type="SimpleTransmission" name="ball_torque">
    <actuator name="torque" />
    <joint name="theta1" />
    <mechanicalReduction>1</mechanicalReduction>
  </transmission>

  <transmission type="SimpleTransmission" name="ball_force">
    <actuator name="force" />
    <joint name="x" />
    <mechanicalReduction>.1</mechanicalReduction>
  </transmission>
</robot>
"""

#here the ball is 20% smaller in radius
ballbot_smallball = """
<?xml version="1.0"?>

<robot xmlns="http://drake.mit.edu"
 xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
 name="BallBot">

  <link name="ground">
    <visual>
      <origin xyz="0 0 -5" rpy="0 0 0" />
      <geometry>
        <box size="1000 1000 10" />
      </geometry>
      <material>
        <color rgba="0.93 .74 .4 1" />
      </material>
    </visual>
  </link>

  <joint name="ground_weld" type="fixed">
    <parent link="world" />
    <child link="ground" />
  </joint>

  <link name="ball">
    <inertial>
      <origin xyz="0 0 0" rpy="0 0 0" />
      <mass value="5" />
      <inertia ixx=".2" ixy="0" ixz="0" iyy="0.2" iyz="0" izz="0.2" />
    </inertial>
    <visual>
      <origin xyz="0 0 0" rpy="0 0 0" />
      <geometry>
        <sphere radius=".08" />
      </geometry>
      <material>
        <color rgba="0.25 0.52 0.96 1" />
      </material>
    </visual>
  </link>

  <link name="bot">
    <inertial>
      <origin xyz="0 0 .05" rpy="0 0 0" />
      <mass value="4" />
      <inertia ixx="0.10" ixy="0" ixz="0" iyy="0.18" iyz="0" izz="0.1" />
    </inertial>
    <visual>
      <origin xyz="0 0 .04" rpy="0 0 0" />
      <geometry>
         <cylinder length=".1" radius=".12" />
      </geometry>
      <material>
        <color rgba=".61 .63 .67 1" />
      </material>
    </visual>
  </link>
  
  <joint name="x" type="prismatic">
    <parent link="world" />
    <child link="ball" />
    <origin xyz="0 0 .1" />
    <axis xyz="1 0 0" />
    <dynamics damping="0.1" />
  </joint>
   

  <joint name="theta1" type="continuous">
    <parent link="ball" />
    <child link="bot" />
    <origin xyz="0 0 0" />
    <axis xyz="0 1 0" />
    <dynamics damping="0.1" />
  </joint>


  <transmission type="SimpleTransmission" name="ball_torque">
    <actuator name="torque" />
    <joint name="theta1" />
    <mechanicalReduction>1</mechanicalReduction>
  </transmission>

  <transmission type="SimpleTransmission" name="ball_force">
    <actuator name="force" />
    <joint name="x" />
    <mechanicalReduction>.08</mechanicalReduction>
  </transmission>
</robot>
"""

Now lets get to the actual FUN part here we'll have a helper function to create the ball bot, then we'll create a few different diagrams to hook it up to different controllers and plot the results!

In [ ]:
#[position X ,theta, xdot, theat_dot, integrator]
start1 = [1,0,0,0,0]
start2 = [-1,np.pi/4,0,0,0]
start3= [-0.5, 1,0,0,0]
x_star = [0,0,0,0]
Q = np.diag((10., 10., 1., 1.))
R = np.eye(1)
timestep = 0.01
Duration = 30 

In [ ]:
#this reutnrs a connected ball bot
def BallBot(urdif):
  builder = DiagramBuilder()
  ballbot, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
  Parser(ballbot).AddModelFromString(urdif, "urdf")
  ballbot.set_name("ballbot")
  ballbot.Finalize()

  B = np.array([[1],[1]])

  gain = builder.AddSystem(pydrake.systems.primitives.MatrixGain(B))
  gain.set_name("Actuator Mapping")

  builder.Connect(gain.get_output_port(),ballbot.get_actuation_input_port())
  builder.ExportInput(gain.get_input_port(),"torque")
  builder.ExportOutput(ballbot.get_state_output_port(), "continuous_state")
  builder.ExportOutput(scene_graph.get_query_output_port(), "query")
  builder.ExportOutput(scene_graph.get_pose_bundle_output_port(), "pose")

  return builder.Build(),scene_graph


In [ ]:
#Running cost thing copied from lecture
@TemplateSystem.define("RunningCost_")
def RunningCost_(T):

    class Impl(LeafSystem_[T]):

        def _construct(self, converter=None):
            LeafSystem_[T].__init__(self, converter)
            self.DeclareVectorInputPort("state", BasicVector_[T](4))
            self.DeclareVectorInputPort("command", BasicVector_[T](1))
            self.DeclareVectorOutputPort("cost", BasicVector_[T](1),self.CostOutput)

        def _construct_copy(self, other, converter=None):
            Impl._construct(self, converter=converter)

        def CostOutput(self, context, output):
            x = self.get_input_port(0).Eval(context)
            u = self.get_input_port(1).Eval(context)
            #print("X",x,"u",u)
            #print("Q",Q)
            #print("R",R)
            #print()
            a = x.T@(Q@(x)) + R.dot(u**2)
            output[0] = a[0]

    return Impl
x = np.array([0,1,1,0])
u = np.array([0])
print((x.dot(Q.dot(x)) + R.dot(u**2)))
RunningCost = RunningCost_[None]  # Default instantiation

LQR Diagram


In [ ]:
#default LQR 
def defaultLQR():
  defaultBallBot,scene_graph = BallBot(ballbot_standard)
  input_i = defaultBallBot.get_input_port().get_index()
  context = defaultBallBot.CreateDefaultContext()
  context.get_mutable_continuous_state_vector().SetFromVector(x_star)
  defaultBallBot.get_input_port().FixValue(context, [0])


  return LinearQuadraticRegulator(defaultBallBot, context, Q, R, input_port_index=input_i)




def buildLQR(urdf):
  builder = DiagramBuilder()

  ballbotsystem,scene_graph = BallBot(urdf)
  ballbotsystem.set_name("System BallBot")
  ballbotsystem = builder.AddSystem(ballbotsystem)



  # set the operating point (vertical unstable equilibrium)
  context = ballbotsystem.CreateDefaultContext()
  context.get_mutable_continuous_state_vector().SetFromVector(x_star)

  # fix the input port to zero and get its index for the lqr function
  ballbotsystem.get_input_port().FixValue(context, [0])
  input_i = ballbotsystem.get_input_port().get_index()
  states = LogOutput(ballbotsystem.get_output_port(0),builder)

  # synthesize lqr controller directly from
  # the nonlinear system and the operating point

  lqr = builder.AddSystem(defaultLQR())
  lqr.set_name("LQR")
  torques = LogOutput(lqr.get_output_port(0),builder)
  # wire cart-pole and lqr
  builder.Connect(ballbotsystem.GetOutputPort("continuous_state"), lqr.get_input_port(0))
  builder.Connect(lqr.get_output_port(0), ballbotsystem.get_input_port())

  #This didn't work so I ignored it
  running_cost = builder.AddSystem(RunningCost())
  running_cost.set_name("running_cost")
  running_cost_logger = LogOutput(running_cost.get_output_port(0),builder)
  builder.Connect(ballbotsystem.GetOutputPort("continuous_state"), running_cost.get_input_port(0))
  builder.Connect(lqr.get_output_port(0), running_cost.get_input_port(1))

  #integrated
  cost = builder.AddSystem(Integrator(1))
  cost.set_name("integrator")
  builder.Connect(running_cost.get_output_port(), cost.get_input_port())
  cost_logger = LogOutput(cost.get_output_port(0),builder)
  #cost = LogOutput(running_cost.get_output_port(),builder)
  # add a visualizer and wire it
  visualizer = builder.AddSystem(
    PlanarSceneGraphVisualizer(scene_graph, xlim=[-3., 2.], ylim=[-.4, 1.5], show=False)
  )


  builder.Connect(ballbotsystem.GetOutputPort("pose"), visualizer.get_input_port(0))

  # finish building the block diagram
  diagram = builder.Build()
  #display(SVG(pydot.graph_from_dot_data(diagram.GetGraphvizString())[0].create_svg()))
  # instantiate a simulator
  simulator = Simulator(diagram)
  simulator.set_publish_every_time_step(False) # makes sim faster
  return context,visualizer,states,torques,running_cost_logger,cost_logger,simulator


In [ ]:
# function that given the cart-pole initial state
# and the simulation time, simulates the system
# and produces a video
def simulate_and_animate(x0, urdf, sim_time=10):
    context, visualizer,states,torques,running_cost,cost, simulator = buildLQR(urdf)
    # start recording the video for the animation of the simulation
    visualizer.start_recording()
    
    # reset initial time and state
    context = simulator.get_mutable_context()
    context.SetTime(0.)
    
    context.SetContinuousState(x0)
    
    


    # run sim
    simulator.Initialize()
    simulator.AdvanceTo(sim_time)
    
    # stop video
    visualizer.stop_recording()
    
    # construct animation
    ani = visualizer.get_recording_as_animation()
    
    # display animation below the cell
    display(HTML(ani.to_jshtml()))
    
    # reset to empty video
    return states,torques,running_cost,cost

In [ ]:
states,torques,running_cost, cost = simulate_and_animate(start1,urdf=ballbot_standard)

fig, ax = plt.subplots(1,2)
fig.set_size_inches(5, 5)
ax[0].plot(running_cost.sample_times(), running_cost.data().T)

c1 = np.stack((cost.sample_times().reshape(359),cost.data().reshape(359)))
rcl = np.stack((running_cost.sample_times().reshape(359),running_cost.data().reshape(359)))
scipy.io.savemat('test.mat', {'cost':c1,'running_cost':rcl})
ax[1].plot(cost.sample_times(), cost.data().T)
print(cost.data().max())

states,torques,running_cost, cost =simulate_and_animate(start1,ballbot_heavyball)
ax[0].plot(running_cost.sample_times(), running_cost.data().T)


ax[1].plot(cost.sample_times(), cost.data().T)

print(cost.data().max())

states,torques,running_cost, cost =simulate_and_animate(start1,ballbot_smallball)
ax[0].plot(running_cost.sample_times(), running_cost.data().T)
ax[0].legend(["Standard","Heavy Ball","Small Ball"])

ax[1].plot(cost.sample_times(), cost.data().T)
ax[1].legend(["Standard","Heavy Ball","Small Ball"])
print(cost.data().max())

ax[0].set_ylabel("Running Cost")
ax[0].set_xlabel("Time (s)")
ax[0].set_title("Running cost")

ax[1].set_xlabel("Cost")
ax[1].set_xlabel("Time (s)")
ax[1].set_title("Cost")


fig.savefig("LQR Start 1",dpi=300)

In [ ]:
a =cost.sample_times().reshape(359)
b=cost.data().reshape(359)



In [ ]:
states,torques,running_cost, cost = simulate_and_animate(start2,urdf=ballbot_standard)

fig, ax = plt.subplots(1,2)
fig.set_size_inches(5, 5)
ax[0].plot(running_cost.sample_times(), running_cost.data().T)

ax[1].plot(cost.sample_times(), cost.data().T)
print(cost.data().max())

states,torques,running_cost, cost =simulate_and_animate(start2,ballbot_heavyball)
ax[0].plot(running_cost.sample_times(), running_cost.data().T)


ax[1].plot(cost.sample_times(), cost.data().T)

states,torques,running_cost, cost =simulate_and_animate(start2,ballbot_smallball)
ax[0].plot(running_cost.sample_times(), running_cost.data().T)
ax[0].legend(["Standard","Heavy Ball","Small Ball"])

ax[1].plot(cost.sample_times(), cost.data().T)
ax[1].legend(["Standard","Heavy Ball","Small Ball"])
fig.savefig("LQR Start 2",dpi=300)

In [ ]:
states,torques,running_cost, cost = simulate_and_animate(start3,urdf=ballbot_standard)

fig, ax = plt.subplots(1,2)
fig.set_size_inches(5, 5)

ax[0].plot(running_cost.sample_times(), running_cost.data().T)


ax[1].plot(cost.sample_times(), cost.data().T)

print(cost.data().max())

states,torques,running_cost, cost =simulate_and_animate(start3,ballbot_heavyball)

ax[0].plot(running_cost.sample_times(), running_cost.data().T)

ax[1].plot(cost.sample_times(), cost.data().T)

print(cost.data().max())

states,torques,running_cost, cost =simulate_and_animate(start3,ballbot_smallball)
ax[0].plot(running_cost.sample_times(), running_cost.data().T)
ax[0].legend(["Standard","Heavy Ball","Small Ball"])

ax[1].plot(cost.sample_times(), cost.data().T)
ax[1].legend(["Standard","Heavy Ball","Small Ball"])
print(cost.data().max())
fig.savefig("LQR Start 3",dpi=300)

Now lets get the NN working "YAY"
I'm going to add abunch of spaces to make this easier to seperate:

//

//

//
..
..
//

NOw we can tell them apart


In [ ]:
#here is the NN
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#Define the function approximator for J


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4,80)
        self.fc2 = nn.Tanh()
        self.fc3 = nn.Linear(80,600)
        self.fc4 = nn.Tanh()
        self.fc5 = nn.Linear(600,1)
      
        #self.fc1 = nn.Linear(2, 1) # placeholder

    def forward(self, x):
        ### TODO ###
        x = self.fc5(self.fc4(self.fc3(self.fc2(self.fc1(x)))))
        return x

net = Net()
net.load_state_dict(torch.load("NetWeightsTest10(1)",map_location=torch.device('cpu')))

In [ ]:
#This is net V2

class Net2(nn.Module):

    def __init__(self):
        super(Net2, self).__init__()
        self.fc1 = nn.Linear(4,80)
        self.fc2 = nn.Tanh()
        self.fc3 = nn.Linear(80,200)
        self.fc4 = nn.Tanh()
        self.fc5 = nn.Linear(200,600)
        self.fc6 = nn.Tanh()
        self.fc7 = nn.Linear(600,1)
 
      
        #self.fc1 = nn.Linear(2, 1) # placeholder

    def forward(self, x):
        ### TODO ###
        x = self.fc7(self.fc6(self.fc5(self.fc4(self.fc3(self.fc2(self.fc1(x)))))))
        return x

net = Net2()
net.load_state_dict(torch.load("NetWeightsTest12",map_location=torch.device('cpu')))

In [ ]:
class Net3(nn.Module):

    def __init__(self):
        super(Net3, self).__init__()
        self.fc1 = nn.Linear(4,80)
        self.fc2 = nn.Tanh()
        self.fc3 = nn.Linear(80,80)
        self.fc4 = nn.Tanh()
        self.fc5 = nn.Linear(80,50)
        self.fc6 = nn.Tanh()
        self.fc7 = nn.Linear(50,1)
 
      
        #self.fc1 = nn.Linear(2, 1) # placeholder

    def forward(self, x):
        ### TODO ###
        x = self.fc7(self.fc6(self.fc5(self.fc4(self.fc3(self.fc2(self.fc1(x)))))))
        return x
net = Net3()
net.load_state_dict(torch.load("NetWeightsTest13",map_location=torch.device('cpu')))       

Here is the the thing that we need for calculating the next step.


In [ ]:
#Calculates the next timestep
#X shold be in shape [# positions][# inputs][1][#states]
#usshold be in shape [# positions][# inputs][1][1]
def CalcNextStep(X,us,timestep):
  theta = X[:,:,:,1].unsqueeze(-1)
  cosq = theta.cos()
  cos2q = cosq.pow(2)
  #cosq2 = (2*theta).cos()
  sinq = theta.sin()
  sin2q = (theta*2).sin()
  V1 = X[:,:,:,3].unsqueeze(-1)
  V12 = V1.pow(2)
  V0 = X[:,:,:,2].unsqueeze(-1)


  a = 4*cos2q - 171
  VD1 = -(.02*(190*sinq*V12+100*cosq*V1 + 950*us-95*V0 -1962*cosq*sinq - 1000*us*cosq)).div(a)
  VD2 = -(0.2*(-20*cosq*sinq*V12-450*V1+4500*us+8829*sinq-100*us*cosq+10*V0*cosq)).div(a)
  dxdt = torch.stack((V0.flatten(2,3),V1.flatten(2,3),VD1.flatten(2,3),VD2.flatten(2,3)),3)
  Xnext=dxdt*timestep + X

  return Xnext

#Quadratic cost

Qt = torch.from_numpy(Q)
Rt = torch.from_numpy(R)
Qt =Qt.type(torch.FloatTensor)
Rt =Rt.type(torch.FloatTensor)
def quadratic_regulator_cost(x,u):
  G = x.matmul(Qt.matmul(x.transpose(-2,-1))) + u@Rt@u.transpose(-2,-1)

  #G = G + x[:,:,:,1].abs().gt(np.pi/4).unsqueeze(-1)*30
  return G

In [ ]:
class Policy(VectorSystem):

    def __init__(self, net, gamma, us):
        # 4 inputs: ball bot state [q, q_dot]
        # 1 output: control torque [u]
        VectorSystem.__init__(self, 4,  1)
        self.net = net
        self.gamma = gamma
        self.us = us # torch tensor with all possible actions
        self.timestep=0.01
        #self.A = torch.tensor(sys.A(),dtype=torch.float)
        #self.B = torch.tensor(sys.B(),dtype = torch.float)
        

    def DoCalcVectorOutput(self, context, state, unused, torque):
        
        X = torch.ones([1,self.us.size()[1],1,4])*torch.tensor(state,dtype=torch.float)

        Xnext1 = CalcNextStep(X,self.us,self.timestep)

        G = quadratic_regulator_cost(X,self.us)*self.timestep

        with torch.no_grad():
          Jnext = self.net.forward(Xnext1)
          
          Jd, ind = torch.min(G + Jnext*0.96, dim=1)
          
        torque[:] = [self.us[0,ind[0],0,0]]

In [ ]:
def simulate_policy(start_state,urdf):
 
  us = torch.linspace(-4,4,200).unsqueeze(0).unsqueeze(-1).unsqueeze(-1)

  # Instantiate the policy system
  # Wire up the drake diagram
  policy = Policy(net, 0.98, us )
  builder = DiagramBuilder() # instantiate a diagram builder
  ballbotsystem,scene_graph = BallBot(urdf)
  ballbotsystem.set_name("System BallBot")
  
  plant = builder.AddSystem(ballbotsystem) # add a sub-system to it
  vi_policy = builder.AddSystem(policy)
  builder.Connect(plant.GetOutputPort("continuous_state"), vi_policy.get_input_port(0)) # connecting inputs/outputs of two sub-systems in our diagram
  builder.Connect(vi_policy.get_output_port(0), plant.get_input_port())
  
  
  states = LogOutput(plant.get_output_port(0),builder)

  torques = LogOutput(vi_policy.get_output_port(0),builder)
  
  visualizer = builder.AddSystem(
    PlanarSceneGraphVisualizer(scene_graph, xlim=[-3., 2.], ylim=[-.4, 1.5], show=False)
  )

  builder.Connect(plant.GetOutputPort("pose"), visualizer.get_input_port(0))




  #ax1.set_ylabel('X pos')
  #ax1.set_xlabel('Time Sec')


  running_cost = builder.AddSystem(RunningCost())
  running_cost.set_name("running_cost")
  running_cost_logger = LogOutput(running_cost.get_output_port(0),builder)
  
  builder.Connect(ballbotsystem.GetOutputPort("continuous_state"), running_cost.get_input_port(0))
  builder.Connect(vi_policy.get_output_port(0), running_cost.get_input_port(1))

  #integrated
  cost = builder.AddSystem(Integrator(1))
  cost.set_name("integrator")
  builder.Connect(running_cost.get_output_port(), cost.get_input_port())
  

  cost_logger = LogOutput(cost.get_output_port(0),builder)
  
  diagram = builder.Build() # finish building the diagram

    # Simulate the system
  simulator = Simulator(diagram)
  simulator.get_mutable_context().SetContinuousState(start_state) # set the initial state
  AdvanceToAndVisualize(simulator, visualizer,10)

  return states,torques,running_cost_logger,cost_logger




In [ ]:

fig, ax = plt.subplots(1,2)
fig.set_size_inches(5, 5)

states,torques,running_cost, cost= simulate_policy(start1,ballbot_standard)


c1 = np.stack((cost.sample_times().reshape(1391 ),cost.data().reshape(1391 )))
rcl = np.stack((running_cost.sample_times().reshape(1391 ),running_cost.data().reshape(1391 )))
scipy.io.savemat('NN.mat', {'cost':c1,'running_cost':rcl})


ax[0].plot(running_cost.sample_times(), running_cost.data().T)


ax[1].plot(cost.sample_times(), cost.data().T)

print(cost.data().max())

states,torques,running_cost, cost =simulate_policy(start1,ballbot_heavyball)


ax[0].plot(running_cost.sample_times(), running_cost.data().T)

ax[1].plot(cost.sample_times(), cost.data().T)

print(cost.data().max())

states,torques,running_cost, cost =simulate_policy(start1,ballbot_smallball)

ax[0].plot(running_cost.sample_times(), running_cost.data().T)
ax[0].legend(["Standard","Heavy Ball","Small Ball"])

ax[1].plot(cost.sample_times(), cost.data().T)
ax[1].legend(["Standard","Heavy Ball","Small Ball"])
print(cost.data().max())


ax[0].set_ylabel("Running Cost")
ax[0].set_xlabel("Time (s)")
ax[0].set_title("Running cost")

ax[1].set_ylabel("Cost")
ax[1].set_xlabel("Time (s)")
ax[1].set_title("Cost")

fig.savefig("NN Value Approximation Start 1",dpi=300)

In [ ]:
fig, ax = plt.subplots(1,2)
fig.set_size_inches(5, 5)

states,torques,running_cost, cost= simulate_policy(start2,ballbot_standard)

ax[0].plot(running_cost.sample_times(), running_cost.data().T)


ax[1].plot(cost.sample_times(), cost.data().T)

print(cost.data().max())

states,torques,running_cost, cost =simulate_policy(start2,ballbot_heavyball)


ax[0].plot(running_cost.sample_times(), running_cost.data().T)

ax[1].plot(cost.sample_times(), cost.data().T)

print(cost.data().max())

states,torques,running_cost, cost =simulate_policy(start2,ballbot_smallball)

ax[0].plot(running_cost.sample_times(), running_cost.data().T)
ax[0].legend(["Standard","Heavy Ball","Small Ball"])

ax[1].plot(cost.sample_times(), cost.data().T)
ax[1].legend(["Standard","Heavy Ball","Small Ball"])
print(cost.data().max())
fig.savefig("NN Value Approximation Start 2",dpi=300)

In [ ]:
fig, ax = plt.subplots(1,2)
fig.set_size_inches(5, 5)

states,torques,running_cost, cost= simulate_policy(start3,ballbot_standard)

ax[0].plot(running_cost.sample_times(), running_cost.data().T)


ax[1].plot(cost.sample_times(), cost.data().T)

print(cost.data().max())

states,torques,running_cost, cost =simulate_policy(start3,ballbot_heavyball)


ax[0].plot(running_cost.sample_times(), running_cost.data().T)

ax[1].plot(cost.sample_times(), cost.data().T)

print(cost.data().max())

states,torques,running_cost, cost =simulate_policy(start3,ballbot_smallball)

ax[0].plot(running_cost.sample_times(), running_cost.data().T)
ax[0].legend(["Standard","Heavy Ball","Small Ball"])

ax[1].plot(cost.sample_times(), cost.data().T)
ax[1].legend(["Standard","Heavy Ball","Small Ball"])
print(cost.data().max())
fig.savefig("NN Value Approximation Start 3",dpi=300)

In [ ]:
fig, ax = plt.subplots(1,2)
fig.set_size_inches(5, 5)

states,torques,running_cost, cost= simulate_policy(start2,ballbot_standard)

ax[0].plot(running_cost.sample_times(), running_cost.data().T)
print(cost.data().max())

ax[1].plot(cost.sample_times(), cost.data().T)
states,torques,running_cost, cost= simulate_and_animate(start2,ballbot_standard)

ax[0].plot(running_cost.sample_times(), running_cost.data().T)


ax[1].plot(cost.sample_times(), cost.data().T)
ax[0].legend(["Dynamic Programming","LQR"])

ax[1].legend(["Dynamic Programming","LQR"])
print(cost.data().max())

ax[0].set_ylabel("Running Cost")
ax[0].set_xlabel("Time (s)")
ax[0].set_title("Running cost")

ax[1].set_ylabel("Cost")
ax[1].set_xlabel("Time (s)")
ax[1].set_title("Cost")


fig.savefig("LQRvsDP",dpi=300)

In [ ]:
#Lets things 
x1s = torch.linspace(-1,1,31)
x2s = torch.linspace(-2,2,51)
x3 = torch.tensor(0,dtype=torch.float);
x4 = torch.tensor(0,dtype=torch.float);
X1s, X2s, X3, X4 = torch.meshgrid(x1s, x2s, x3,x4)

X = torch.stack((X1s.flatten(),X2s.flatten(),X3.flatten(),X4.flatten()),1).unsqueeze(1).unsqueeze(1)
print(X.size())

In [ ]:

def quadratic_regulator_solution(timestep):
  ballbotsystem,scene_graph = BallBot(ballbot_standard)
  context = ballbotsystem.CreateDefaultContext()
  context.get_mutable_continuous_state_vector().SetFromVector(x_star)
  ballbotsystem.get_input_port().FixValue(context, [0])
  sys = pydrake.systems.primitives.FirstOrderTaylorApproximation(ballbotsystem, context)

  S = DiscreteAlgebraicRiccatiEquation(sys.A()*timestep+np.eye(4),sys.B()*timestep,
                                       Q=Q, R=R)
  
  return S


S = torch.from_numpy(quadratic_regulator_solution(0.01)).float()

In [ ]:
J = net.forward(X).flatten(-3,-1)

J2 = X@S@X.transpose(-2,-1)

In [ ]:
from matplotlib import cm
fig = plt.figure(figsize=(9, 4))
ax1, ax2 = fig.subplots(1, 2, subplot_kw=dict(projection='3d'))


ax1.set_xlabel("x")
ax1.set_ylabel("theta")
ax1.set_title("Estimated Cost-to-Go")

test1 = J.view(X1s.size()).flatten(-3,-1).detach().numpy()

ax1.plot_surface(X1s.flatten(-3,-1), X2s.flatten(-3,-1), test1,  cmap=cm.jet)


ax2.set_xlabel("x")
ax2.set_ylabel("theta")
ax2.set_title("LQR Cost-to-Go")

test2 = J2.view(X1s.size()).flatten(-3,-1).detach().numpy()

ax2.plot_surface(X1s.flatten(-3,-1), X2s.flatten(-3,-1), test2,  cmap=cm.jet)
criterion = nn.MSELoss()
score = criterion(J, J2).item()
print("MSE(Ĵᵢ,Jᵢ) = %.2f" % score) 
fig.savefig("EstimatedCosttoGO",dpi=300)